# Machine Learning using datasets in Group A

Data reflective of those of rail operations. LDPRF 2097 is used for training while LDPRF 2098 is used for testing.
Systematic reduction of datasets in 10% increments

### Import necessary libraries

In [1]:
import numpy as np
import pandas as pd
import copy
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# import codebase
import thermalModel_main as tmm
import thermalModel_groupB as tm_gb

import importlib
importlib.reload(tmm)
importlib.reload(tm_gb)

Using TensorFlow backend.


<module 'thermalModel_groupB' from 'C:\\Users\\user\\Anaconda3\\lib\\thermalModel_groupB.py'>

## ANN Ah Model

### Data loading and cleaning

In [2]:
df = tm_gb.load_csv(filename = 'LDPRF_2097.csv', 
#                     data_list = ['Program time','AhCha','AhDch','Temp'], 
                    features_list = ['runtime_s','AhCha','AhDch','Amb','Temp'], 
                    mode = 0)

df1 = tm_gb.load_csv(filename = 'LDPRF_2098.csv', 
#                      data_list = ['Program time','AhCha','AhDch','Temp'], 
                     features_list = ['runtime_s','AhCha','AhDch','Amb','Temp'], 
                     mode = 0)

C:\Users\user\Anaconda3\lib\thermalModel_groupB.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['second'][set_index[index]:set_index[index+1]] = df['second'][set_index[index]:set_index[index+1]] + second_increment[index]
C:\Users\user\Anaconda3\lib\thermalModel_groupB.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['second'][set_index[index]:] = df['second'][set_index[index]:] + second_increment[index]
C:\Users\user\Anaconda3\lib\thermalModel_groupB.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.

In [3]:
ANN_Ah_models_2097 = {}
ANN_Ah_me_2097 = {}

sections_list = [round(i, 2) for i in np.linspace(start = 0.9, stop = 0.1, num = 9, endpoint = True)]
num_rows = df.shape[0]

for i in range(len(sections_list)):
    boundary = int(num_rows * sections_list[i])
    reduced_df = df[:boundary].copy(deep=True)
    
    reduced_df.drop(columns = ['runtime_s'], inplace = True)
    try:
        df1.drop(columns = ['runtime_s'], inplace = True)
    except:
        pass
    
    print(reduced_df.describe())
    print(df1.describe())
    
    Ah_models_2097, Ah_me_2097 = tmm.loop_run_instances(identifier = "ANN" + '_' + str(sections_list[i]), 
                                                           loop_name = "Ah_model", 
                                                           num_layers = 1, 
                                                           train_dataframe = reduced_df, 
                                                           test_dataframe = df1, 
                                                           num_inputs = 3, 
                                                           start_window_size = 1, 
                                                           end_window_size = 1, 
                                                           window_size_step = 1, 
                                                           test_size = 0, 
                                                           num_epochs = 1000)
    
    ANN_Ah_models_2097["ANN" + '_' + str(sections_list[i])] = Ah_models_2097
    ANN_Ah_me_2097["ANN" + '_' + str(sections_list[i])] = copy.deepcopy(Ah_me_2097)

               AhCha          AhDch           Amb           Temp
count  392255.000000  392255.000000  3.922550e+05  392255.000000
mean      113.702847     131.687044  2.579465e+01      34.137644
std        65.565231      67.188310  2.212207e-10       2.093237
min         0.000000       0.000000  2.579465e+01      25.794650
25%        57.448000      73.137500  2.579465e+01      32.352610
50%       113.693000     130.609000  2.579465e+01      34.975800
75%       170.261000     190.321000  2.579465e+01      35.740890
max       228.876000     248.523000  2.579465e+01      36.615290
               AhCha          AhDch           Amb           Temp
count  435839.000000  435839.000000  4.358390e+05  435839.000000
mean      126.437695     143.968215  2.626750e+01      34.934164
std        72.927347      74.268447  6.600594e-11       1.938317
min         0.000000       0.000000  2.626750e+01      26.267500
25%        64.531000      80.996500  2.626750e+01      33.803260
50%       126.152000     

max        74.972000      92.787000  2.579465e+01      34.101400
               AhCha          AhDch           Amb           Temp
count  435839.000000  435839.000000  4.358390e+05  435839.000000
mean      126.437695     143.968215  2.626750e+01      34.934164
std        72.927347      74.268447  6.600594e-11       1.938317
min         0.000000       0.000000  2.626750e+01      26.267500
25%        64.531000      80.996500  2.626750e+01      33.803260
50%       126.152000     144.421000  2.626750e+01      35.741030
75%       188.089000     207.443000  2.626750e+01      36.386950
max       252.044000     270.765000  2.626750e+01      37.032870
Run parameters: 1_[3]_relu_earlyStop
Restoring model weights from the end of the best epoch
Epoch 00092: early stopping
Time to train model: 399.1638240814209 seconds
              AhCha         AhDch           Amb          Temp
count  87167.000000  87167.000000  8.716700e+04  87167.000000
mean      25.596328     41.319628  2.579465e+01     30.8697

In [4]:
importlib.reload(tmm)
importlib.reload(tm_gb)
ANN_Ah_models_2097_df = tm_gb.extract_complexity(nested_model_dictionary = ANN_Ah_models_2097, 
                                                 nested_errors_dictionary = ANN_Ah_me_2097)

In [5]:
ANN_Ah_models_2097_df

,Percentage_reduced,NN_size,mean_error
0,10,16,0.036776
1,20,16,0.088432
2,30,16,-0.577850
3,40,16,-0.757336
4,50,16,0.481733
5,60,16,-1.755333
6,70,16,-9.130228
7,80,16,3.022391
8,90,16,-2.007231


## ANN IV Model

### Data loading and cleaning

In [6]:
df = tm_gb.load_csv(filename = 'LDPRF_2097.csv', 
#                     data_list = ['Program time','AhCha','AhDch','Temp'], 
                    features_list = ['runtime_s','Current','Voltage','Amb','Temp'], 
                    mode = 1)

df1 = tm_gb.load_csv(filename = 'LDPRF_2098.csv', 
#                      data_list = ['Program time','AhCha','AhDch','Temp'], 
                     features_list = ['runtime_s','Current','Voltage','Amb','Temp'], 
                     mode = 1)

In [7]:
ANN_IV_models_2097 = {}
ANN_IV_me_2097 = {}

sections_list = [round(i, 2) for i in np.linspace(start = 0.9, stop = 0.1, num = 9, endpoint = True)]
num_rows = df.shape[0]

for i in range(len(sections_list)):
    boundary = int(num_rows * sections_list[i])
    reduced_df = df[:boundary].copy(deep=True)
    
    reduced_df.drop(columns = ['runtime_s'], inplace = True)
    try:
        df1.drop(columns = ['runtime_s'], inplace = True)
    except:
        pass
    
    print(reduced_df.describe())
    print(df1.describe())
    
    IV_models_2097, IV_me_2097 = tmm.loop_run_instances(identifier = "ANN" + '_' + str(sections_list[i]), 
                                                           loop_name = "IV_model", 
                                                           num_layers = 1, 
                                                           train_dataframe = reduced_df, 
                                                           test_dataframe = df1, 
                                                           num_inputs = 3, 
                                                           start_window_size = 1, 
                                                           end_window_size = 1, 
                                                           window_size_step = 1, 
                                                           test_size = 0, 
                                                           num_epochs = 1000)
    
    ANN_IV_models_2097["ANN" + '_' + str(sections_list[i])] = IV_models_2097
    ANN_IV_me_2097["ANN" + '_' + str(sections_list[i])] = copy.deepcopy(IV_me_2097)

             Current        Voltage           Amb           Temp
count  392255.000000  392255.000000  3.922550e+05  392255.000000
mean       -0.604674       3.779421  2.579465e+01      34.137644
std        86.197685       0.091355  2.212207e-10       2.093237
min      -177.304100       3.543280  2.579465e+01      25.794650
25%         0.009580       3.736000  2.579465e+01      32.352610
50%         0.009580       3.771040  2.579465e+01      34.975800
75%         0.019150       3.810510  2.579465e+01      35.740890
max       222.608110       4.160100  2.579465e+01      36.615290
             Current        Voltage           Amb           Temp
count  435839.000000  435839.000000  4.358390e+05  435839.000000
mean       -0.497548       3.782469  2.626750e+01      34.934164
std        85.732075       0.086605  6.600594e-11       1.938317
min      -176.603480       3.557440  2.626750e+01      26.267500
25%         0.009560       3.741410  2.626750e+01      33.803260
50%         0.009560     

max       219.878550       4.160100  2.579465e+01      34.101400
             Current        Voltage           Amb           Temp
count  435839.000000  435839.000000  4.358390e+05  435839.000000
mean       -0.497548       3.782469  2.626750e+01      34.934164
std        85.732075       0.086605  6.600594e-11       1.938317
min      -176.603480       3.557440  2.626750e+01      26.267500
25%         0.009560       3.741410  2.626750e+01      33.803260
50%         0.009560       3.773560  2.626750e+01      35.741030
75%         0.009560       3.813390  2.626750e+01      36.386950
max       222.893370       4.161120  2.626750e+01      37.032870
Run parameters: 1_[3]_relu_earlyStop
Restoring model weights from the end of the best epoch
Epoch 00006: early stopping
Time to train model: 26.20138168334961 seconds
            Current       Voltage           Amb          Temp
count  87167.000000  87167.000000  8.716700e+04  87167.000000
mean      -0.828209      3.812962  2.579465e+01     30.8697

## ANN Hybrid Model

### Data loading and cleaning

In [8]:
df = tm_gb.load_csv(filename = 'LDPRF_2097.csv', 
#                     data_list = ['Program time','Current','Voltage','AhCha','AhDch','Temp'], 
                    features_list = ['runtime_s','Current','Voltage','AhCha','AhDch','Amb','Temp'], 
                    mode = 2)

df1 = tm_gb.load_csv(filename = 'LDPRF_2098.csv', 
#                      data_list = ['Program time','Current','Voltage','AhCha','AhDch','Temp'], 
                     features_list = ['runtime_s','Current','Voltage','AhCha','AhDch','Amb','Temp'], 
                     mode = 2)

In [9]:
ANN_hybrid_models_2097 = {}
ANN_hybrid_me_2097 = {}

sections_list = [round(i, 2) for i in np.linspace(start = 0.9, stop = 0.1, num = 9, endpoint = True)]
num_rows = df.shape[0]

for i in range(len(sections_list)):
    boundary = int(num_rows * sections_list[i])
    reduced_df = df[:boundary].copy(deep=True)
    
    reduced_df.drop(columns = ['runtime_s'], inplace = True)
    try:
        df1.drop(columns = ['runtime_s'], inplace = True)
    except:
        pass
    
    print(reduced_df.describe())
    print(df1.describe())
    
    hybrid_models_2097, hybrid_me_2097 = tmm.loop_run_instances(identifier = "ANN" + '_' + str(sections_list[i]), 
                                                                   loop_name = "hybrid_model", 
                                                                   num_layers = 1, 
                                                                   train_dataframe = reduced_df, 
                                                                   test_dataframe = df1, 
                                                                   num_inputs = 5, 
                                                                   start_window_size = 1, 
                                                                   end_window_size = 1, 
                                                                   window_size_step = 1, 
                                                                   test_size = 0, 
                                                                   num_epochs = 1000)
    
    ANN_hybrid_models_2097["ANN" + '_' + str(sections_list[i])] = hybrid_models_2097
    ANN_hybrid_me_2097["ANN" + '_' + str(sections_list[i])] = copy.deepcopy(hybrid_me_2097)

             Current        Voltage          AhCha          AhDch  \
count  392255.000000  392255.000000  392255.000000  392255.000000   
mean       -0.604674       3.779421     113.702847     131.687044   
std        86.197685       0.091355      65.565231      67.188310   
min      -177.304100       3.543280       0.000000       0.000000   
25%         0.009580       3.736000      57.448000      73.137500   
50%         0.009580       3.771040     113.693000     130.609000   
75%         0.019150       3.810510     170.261000     190.321000   
max       222.608110       4.160100     228.876000     248.523000   

                Amb           Temp  
count  3.922550e+05  392255.000000  
mean   2.579465e+01      34.137644  
std    2.212207e-10       2.093237  
min    2.579465e+01      25.794650  
25%    2.579465e+01      32.352610  
50%    2.579465e+01      34.975800  
75%    2.579465e+01      35.740890  
max    2.579465e+01      36.615290  
             Current        Voltage          

max    2.626750e+01      37.032870  
Run parameters: 1_[5]_relu_earlyStop
Restoring model weights from the end of the best epoch
Epoch 00046: early stopping
Time to train model: 134.17318606376648 seconds
            Current       Voltage         AhCha         AhDch           Amb  \
count  43583.000000  43583.000000  43583.000000  43583.000000  4.358300e+04   
mean      -2.458174      3.816734     13.134120     28.477074  2.579465e+01   
std       85.260487      0.096784      7.658945      7.765900  2.755872e-11   
min     -176.375000      3.562210      0.000000      0.000000  2.579465e+01   
25%        0.009580      3.788150      6.916000     21.094500  2.579465e+01   
50%        0.009580      3.803660     13.864000     29.211000  2.579465e+01   
75%        0.009580      3.840310     20.825000     37.092000  2.579465e+01   
max      219.294320      4.160100     24.306000     40.684000  2.579465e+01   

               Temp  
count  43583.000000  
mean      30.348772  
std        1.5011

## ANN errors

In [10]:
importlib.reload(tmm)
importlib.reload(tm_gb)

ANN_Ah_models_2097_df = tm_gb.extract_complexity(nested_model_dictionary = ANN_Ah_models_2097, 
                                                 nested_errors_dictionary = ANN_Ah_me_2097)

ANN_IV_models_2097_df = tm_gb.extract_complexity(nested_model_dictionary = ANN_IV_models_2097, 
                                                 nested_errors_dictionary = ANN_IV_me_2097)

ANN_hybrid_models_2097_df = tm_gb.extract_complexity(nested_model_dictionary = ANN_hybrid_models_2097, 
                                                     nested_errors_dictionary = ANN_hybrid_me_2097)

In [11]:
ANN_reductions_dict = {
    'Ah_model':ANN_Ah_models_2097_df,
    'IV_model':ANN_IV_models_2097_df,
    'hybrid_model':ANN_hybrid_models_2097_df
}

In [12]:
import pickle

with open('ANN_%reductions_dict.pickle', 'wb') as handle:
    pickle.dump(ANN_reductions_dict, handle)

for key in ANN_reductions_dict.keys():
    ANN_reductions_dict[key].to_csv('ANN_%reductions_' + key + '.csv', index=False)

## DNN Ah Model

### Data loading and cleaning

In [13]:
df = tm_gb.load_csv(filename = 'LDPRF_2097.csv', 
#                     data_list = ['Program time','AhCha','AhDch','Temp'], 
                    features_list = ['runtime_s','AhCha','AhDch','Amb','Temp'], 
                    mode = 0)

df1 = tm_gb.load_csv(filename = 'LDPRF_2098.csv', 
#                      data_list = ['Program time','AhCha','AhDch','Temp'], 
                     features_list = ['runtime_s','AhCha','AhDch','Amb','Temp'], 
                     mode = 0)

In [14]:
DNN_Ah_models_2097 = {}
DNN_Ah_me_2097 = {}

sections_list = [round(i, 2) for i in np.linspace(start = 0.9, stop = 0.1, num = 9, endpoint = True)]
num_rows = df.shape[0]

for i in range(len(sections_list)):
    boundary = int(num_rows * sections_list[i])
    reduced_df = df[:boundary].copy(deep=True)
    
    reduced_df.drop(columns = ['runtime_s'], inplace = True)
    try:
        df1.drop(columns = ['runtime_s'], inplace = True)
    except:
        pass
    
    print(reduced_df.describe())
    print(df1.describe())
    
    Ah_models_2097, Ah_me_2097 = tmm.loop_run_instances(identifier = "DNN" + '_' + str(sections_list[i]), 
                                                           loop_name = "Ah_model", 
                                                           num_layers = 2, 
                                                           train_dataframe = reduced_df, 
                                                           test_dataframe = df1, 
                                                           num_inputs = 3, 
                                                           start_window_size = 1, 
                                                           end_window_size = 1, 
                                                           window_size_step = 1, 
                                                           test_size = 0, 
                                                           num_epochs = 1000)
    
    DNN_Ah_models_2097["DNN" + '_' + str(sections_list[i])] = Ah_models_2097
    DNN_Ah_me_2097["DNN" + '_' + str(sections_list[i])] = copy.deepcopy(Ah_me_2097)

               AhCha          AhDch           Amb           Temp
count  392255.000000  392255.000000  3.922550e+05  392255.000000
mean      113.702847     131.687044  2.579465e+01      34.137644
std        65.565231      67.188310  2.212207e-10       2.093237
min         0.000000       0.000000  2.579465e+01      25.794650
25%        57.448000      73.137500  2.579465e+01      32.352610
50%       113.693000     130.609000  2.579465e+01      34.975800
75%       170.261000     190.321000  2.579465e+01      35.740890
max       228.876000     248.523000  2.579465e+01      36.615290
               AhCha          AhDch           Amb           Temp
count  435839.000000  435839.000000  4.358390e+05  435839.000000
mean      126.437695     143.968215  2.626750e+01      34.934164
std        72.927347      74.268447  6.600594e-11       1.938317
min         0.000000       0.000000  2.626750e+01      26.267500
25%        64.531000      80.996500  2.626750e+01      33.803260
50%       126.152000     

max        74.972000      92.787000  2.579465e+01      34.101400
               AhCha          AhDch           Amb           Temp
count  435839.000000  435839.000000  4.358390e+05  435839.000000
mean      126.437695     143.968215  2.626750e+01      34.934164
std        72.927347      74.268447  6.600594e-11       1.938317
min         0.000000       0.000000  2.626750e+01      26.267500
25%        64.531000      80.996500  2.626750e+01      33.803260
50%       126.152000     144.421000  2.626750e+01      35.741030
75%       188.089000     207.443000  2.626750e+01      36.386950
max       252.044000     270.765000  2.626750e+01      37.032870
Run parameters: 1_[3, 3]_relu_earlyStop
Restoring model weights from the end of the best epoch
Epoch 00216: early stopping
Time to train model: 982.3052158355713 seconds
              AhCha         AhDch           Amb          Temp
count  87167.000000  87167.000000  8.716700e+04  87167.000000
mean      25.596328     41.319628  2.579465e+01     30.8

## DNN IV Model

### Data loading and cleaning

In [15]:
df = tm_gb.load_csv(filename = 'LDPRF_2097.csv', 
#                     data_list = ['Program time','AhCha','AhDch','Temp'], 
                    features_list = ['runtime_s','Current','Voltage','Amb','Temp'], 
                    mode = 1)

df1 = tm_gb.load_csv(filename = 'LDPRF_2098.csv', 
#                      data_list = ['Program time','AhCha','AhDch','Temp'], 
                     features_list = ['runtime_s','Current','Voltage','Amb','Temp'], 
                     mode = 1)

In [16]:
DNN_IV_models_2097 = {}
DNN_IV_me_2097 = {}

sections_list = [round(i, 2) for i in np.linspace(start = 0.9, stop = 0.1, num = 9, endpoint = True)]
num_rows = df.shape[0]

for i in range(len(sections_list)):
    boundary = int(num_rows * sections_list[i])
    reduced_df = df[:boundary].copy(deep=True)
    
    reduced_df.drop(columns = ['runtime_s'], inplace = True)
    try:
        df1.drop(columns = ['runtime_s'], inplace = True)
    except:
        pass
    
    print(reduced_df.describe())
    print(df1.describe())
    
    IV_models_2097, IV_me_2097 = tmm.loop_run_instances(identifier = "DNN" + '_' + str(sections_list[i]), 
                                                           loop_name = "IV_model", 
                                                           num_layers = 2, 
                                                           train_dataframe = reduced_df, 
                                                           test_dataframe = df1, 
                                                           num_inputs = 3, 
                                                           start_window_size = 1, 
                                                           end_window_size = 1, 
                                                           window_size_step = 1, 
                                                           test_size = 0, 
                                                           num_epochs = 1000)
    
    DNN_IV_models_2097["DNN" + '_' + str(sections_list[i])] = IV_models_2097
    DNN_IV_me_2097["DNN" + '_' + str(sections_list[i])] = copy.deepcopy(IV_me_2097)

             Current        Voltage           Amb           Temp
count  392255.000000  392255.000000  3.922550e+05  392255.000000
mean       -0.604674       3.779421  2.579465e+01      34.137644
std        86.197685       0.091355  2.212207e-10       2.093237
min      -177.304100       3.543280  2.579465e+01      25.794650
25%         0.009580       3.736000  2.579465e+01      32.352610
50%         0.009580       3.771040  2.579465e+01      34.975800
75%         0.019150       3.810510  2.579465e+01      35.740890
max       222.608110       4.160100  2.579465e+01      36.615290
             Current        Voltage           Amb           Temp
count  435839.000000  435839.000000  4.358390e+05  435839.000000
mean       -0.497548       3.782469  2.626750e+01      34.934164
std        85.732075       0.086605  6.600594e-11       1.938317
min      -176.603480       3.557440  2.626750e+01      26.267500
25%         0.009560       3.741410  2.626750e+01      33.803260
50%         0.009560     

max       219.878550       4.160100  2.579465e+01      34.101400
             Current        Voltage           Amb           Temp
count  435839.000000  435839.000000  4.358390e+05  435839.000000
mean       -0.497548       3.782469  2.626750e+01      34.934164
std        85.732075       0.086605  6.600594e-11       1.938317
min      -176.603480       3.557440  2.626750e+01      26.267500
25%         0.009560       3.741410  2.626750e+01      33.803260
50%         0.009560       3.773560  2.626750e+01      35.741030
75%         0.009560       3.813390  2.626750e+01      36.386950
max       222.893370       4.161120  2.626750e+01      37.032870
Run parameters: 1_[3, 3]_relu_earlyStop
Restoring model weights from the end of the best epoch
Epoch 00045: early stopping
Time to train model: 207.7735366821289 seconds
            Current       Voltage           Amb          Temp
count  87167.000000  87167.000000  8.716700e+04  87167.000000
mean      -0.828209      3.812962  2.579465e+01     30.8

## DNN Hybrid Model

### Data loading and cleaning

In [17]:
df = tm_gb.load_csv(filename = 'LDPRF_2097.csv', 
#                     data_list = ['Program time','Current','Voltage','AhCha','AhDch','Temp'], 
                    features_list = ['runtime_s','Current','Voltage','AhCha','AhDch','Amb','Temp'], 
                    mode = 2)

df1 = tm_gb.load_csv(filename = 'LDPRF_2098.csv', 
#                      data_list = ['Program time','Current','Voltage','AhCha','AhDch','Temp'], 
                     features_list = ['runtime_s','Current','Voltage','AhCha','AhDch','Amb','Temp'], 
                     mode = 2)

In [18]:
DNN_hybrid_models_2097 = {}
DNN_hybrid_me_2097 = {}

sections_list = [round(i, 2) for i in np.linspace(start = 0.9, stop = 0.1, num = 9, endpoint = True)]
num_rows = df.shape[0]

for i in range(len(sections_list)):
    boundary = int(num_rows * sections_list[i])
    reduced_df = df[:boundary].copy(deep=True)
    
    reduced_df.drop(columns = ['runtime_s'], inplace = True)
    try:
        df1.drop(columns = ['runtime_s'], inplace = True)
    except:
        pass
    
    print(reduced_df.describe())
    print(df1.describe())
    
    hybrid_models_2097, hybrid_me_2097 = tmm.loop_run_instances(identifier = "DNN" + '_' + str(sections_list[i]), 
                                                                   loop_name = "hybrid_model", 
                                                                   num_layers = 2, 
                                                                   train_dataframe = reduced_df, 
                                                                   test_dataframe = df1, 
                                                                   num_inputs = 5, 
                                                                   start_window_size = 1, 
                                                                   end_window_size = 1, 
                                                                   window_size_step = 1, 
                                                                   test_size = 0, 
                                                                   num_epochs = 1000)
    
    DNN_hybrid_models_2097["DNN" + '_' + str(sections_list[i])] = hybrid_models_2097
    DNN_hybrid_me_2097["DNN" + '_' + str(sections_list[i])] = copy.deepcopy(hybrid_me_2097)

             Current        Voltage          AhCha          AhDch  \
count  392255.000000  392255.000000  392255.000000  392255.000000   
mean       -0.604674       3.779421     113.702847     131.687044   
std        86.197685       0.091355      65.565231      67.188310   
min      -177.304100       3.543280       0.000000       0.000000   
25%         0.009580       3.736000      57.448000      73.137500   
50%         0.009580       3.771040     113.693000     130.609000   
75%         0.019150       3.810510     170.261000     190.321000   
max       222.608110       4.160100     228.876000     248.523000   

                Amb           Temp  
count  3.922550e+05  392255.000000  
mean   2.579465e+01      34.137644  
std    2.212207e-10       2.093237  
min    2.579465e+01      25.794650  
25%    2.579465e+01      32.352610  
50%    2.579465e+01      34.975800  
75%    2.579465e+01      35.740890  
max    2.579465e+01      36.615290  
             Current        Voltage          

max    2.626750e+01      37.032870  
Run parameters: 1_[5, 5]_relu_earlyStop
Restoring model weights from the end of the best epoch
Epoch 00057: early stopping
Time to train model: 174.63523507118225 seconds
            Current       Voltage         AhCha         AhDch           Amb  \
count  43583.000000  43583.000000  43583.000000  43583.000000  4.358300e+04   
mean      -2.458174      3.816734     13.134120     28.477074  2.579465e+01   
std       85.260487      0.096784      7.658945      7.765900  2.755872e-11   
min     -176.375000      3.562210      0.000000      0.000000  2.579465e+01   
25%        0.009580      3.788150      6.916000     21.094500  2.579465e+01   
50%        0.009580      3.803660     13.864000     29.211000  2.579465e+01   
75%        0.009580      3.840310     20.825000     37.092000  2.579465e+01   
max      219.294320      4.160100     24.306000     40.684000  2.579465e+01   

               Temp  
count  43583.000000  
mean      30.348772  
std        1.5

## DNN errors

In [19]:
importlib.reload(tmm)
importlib.reload(tm_gb)

DNN_Ah_models_2097_df = tm_gb.extract_complexity(nested_model_dictionary = DNN_Ah_models_2097, 
                                                 nested_errors_dictionary = DNN_Ah_me_2097)

DNN_IV_models_2097_df = tm_gb.extract_complexity(nested_model_dictionary = DNN_IV_models_2097, 
                                                 nested_errors_dictionary = DNN_IV_me_2097)

DNN_hybrid_models_2097_df = tm_gb.extract_complexity(nested_model_dictionary = DNN_hybrid_models_2097, 
                                                     nested_errors_dictionary = DNN_hybrid_me_2097)

In [20]:
DNN_reductions_dict = {
    'Ah_model':DNN_Ah_models_2097_df,
    'IV_model':DNN_IV_models_2097_df,
    'hybrid_model':DNN_hybrid_models_2097_df
}

In [21]:
import pickle

with open('DNN_%reductions_dict.pickle', 'wb') as handle:
    pickle.dump(DNN_reductions_dict, handle)

for key in DNN_reductions_dict.keys():
    DNN_reductions_dict[key].to_csv('DNN_%reductions_' + key + '.csv', index=False)